In [1]:
%pip install pandas
%pip install sqlalchemy
%pip install python-dotenv
%pip install psycopg2-binary

# Import des bibliothèques nécessaires
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Chargement des variables d'environnement
load_dotenv()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


True

In [2]:
DB_HOST = 'localhost'
DB_PORT = os.getenv('POSTGRES_PORT')
DB_NAME = os.getenv('POSTGRES_DB') 
DB_USER = os.getenv('POSTGRES_USER') 
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')

# Création de l'URL de connexion
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = create_engine(DATABASE_URL)

In [4]:
# Requêtes d'analyse de la production

# 1. Vue d'ensemble des centres d'usinage
query1 = """
SELECT 
    cu.nom,
    COUNT(DISTINCT sp.id) as nombre_sessions,
    SUM(sp.total_pieces) as total_pieces_produites,
    ROUND(AVG(sp.taux_occupation)::numeric, 2) as taux_occupation_moyen
FROM centre_usinage cu
LEFT JOIN session_production sp ON cu.id = sp.centre_usinage_id
GROUP BY cu.id, cu.nom, cu.type_cu
ORDER BY total_pieces_produites DESC;
"""

print("=== Vue d'ensemble des centres d'usinage ===")
df1 = pd.read_sql(query1, engine)
df1


=== Vue d'ensemble des centres d'usinage ===


,nom,nombre_sessions,total_pieces_produites,taux_occupation_moyen
0,SU12,20,16174,71.47
1,DEM12,20,9591,59.88
2,DEMALU,20,6670,62.74


In [5]:
# 2. Analyse de la production par jour
query2 = """
SELECT 
    sp.date_production,
    cu.nom as centre_usinage,
    SUM(sp.total_pieces) as pieces_produites,
    ROUND(AVG(sp.taux_occupation), 2) as taux_occupation_moyen,
    ROUND(AVG(sp.taux_attente), 2) as taux_attente_moyen
FROM session_production sp
LEFT JOIN centre_usinage cu ON cu.id = sp.centre_usinage_id
GROUP BY sp.date_production, cu.nom
ORDER BY sp.date_production DESC
LIMIT 30;
"""

print("\n=== Analyse de la production par jour (30 derniers jours) ===")
df2 = pd.read_sql(query2, engine)
df2


=== Analyse de la production par jour (30 derniers jours) ===


,date_production,centre_usinage,pieces_produites,taux_occupation_moyen,taux_attente_moyen
0,2025-05-28,DEMALU,44,51.28,13.01
1,2025-05-28,SU12,452,77.18,14.76
2,2025-05-27,DEM12,762,66.73,18.09
3,2025-05-27,DEMALU,301,50.43,11.80
4,2025-05-27,SU12,914,67.52,16.51
5,2025-05-26,DEM12,745,62.30,17.45
6,2025-05-26,DEMALU,462,62.45,13.11
7,2025-05-26,SU12,753,67.98,16.87
8,2025-05-24,DEM12,319,63.15,36.20
9,2025-05-24,DEMALU,265,90.54,0.80


In [6]:
# 3. Analyse des commandes de volets roulants
query3 = """
SELECT 
    status,
    COUNT(*) as nombre_commandes,
    MIN(date_modification) as premiere_commande,
    MAX(date_modification) as derniere_commande,
    SUM(gestion_en_stock) as total_en_stock
FROM commandes_volets_roulants
WHERE status IS NOT NULL
GROUP BY status
ORDER BY nombre_commandes DESC;
"""

print("\n=== Analyse des commandes de volets roulants par status ===")
df3 = pd.read_sql(query3, engine)
df3


=== Analyse des commandes de volets roulants par status ===


,status,nombre_commandes,premiere_commande,derniere_commande,total_en_stock
0,cde Planifiee,23,2025-06-19,2025-07-02,21
1,lancer en prod,7,2025-05-14,2025-07-01,0
2,vitrage,6,2025-06-06,2025-07-01,0


In [7]:
# 4. Analyse des périodes d'arrêt par centre d'usinage
query4 = """
WITH ArretStats AS (
    SELECT 
        cu.nom as centre_usinage,
        sp.date_production,
        COUNT(pa.id) as nombre_arrets,
        ROUND(AVG(pa.duree_secondes)::numeric, 2) as duree_moyenne_arret_sec,
        SUM(pa.duree_secondes) as duree_totale_arrets_sec
    FROM session_production sp
    JOIN centre_usinage cu ON sp.centre_usinage_id = cu.id
    JOIN periode_arret pa ON sp.id = pa.session_id
    GROUP BY cu.nom, sp.date_production
)
SELECT 
    centre_usinage,
    COUNT(*) as jours_avec_arrets,
    ROUND(AVG(nombre_arrets)::numeric, 2) as moyenne_arrets_par_jour,
    ROUND(AVG(duree_moyenne_arret_sec)::numeric, 2) as duree_moyenne_arret_sec,
    ROUND(SUM(duree_totale_arrets_sec) / 3600.0, 2) as duree_totale_arrets_heures
FROM ArretStats
GROUP BY centre_usinage
ORDER BY duree_totale_arrets_heures DESC;
"""

print("\n=== Analyse des périodes d'arrêt par centre d'usinage ===")
df4 = pd.read_sql(query4, engine)
df4


=== Analyse des périodes d'arrêt par centre d'usinage ===


,centre_usinage,jours_avec_arrets,moyenne_arrets_par_jour,duree_moyenne_arret_sec,duree_totale_arrets_heures
0,DEMALU,20,15.70,935.43,67.81
1,DEM12,20,17.80,856.34,67.63
2,SU12,20,25.95,272.31,37.78


In [8]:
# 5. Analyse des jobs par profil
query5 = """
SELECT 
    jp.reference,
    jp.couleur,
    COUNT(*) as nombre_jobs,
    ROUND(AVG(jp.longueur)::numeric, 2) as longueur_moyenne,
    COUNT(DISTINCT sp.id) as nombre_sessions,
    COUNT(DISTINCT sp.centre_usinage_id) as nombre_centres_usinage
FROM job_profil jp
JOIN session_production sp ON jp.session_id = sp.id
GROUP BY jp.reference, jp.couleur
HAVING COUNT(*) > 1
ORDER BY nombre_jobs DESC
LIMIT 15;
"""

print("\n=== Top 15 des profils de jobs les plus fréquents ===")
df5 = pd.read_sql(query5, engine)
df5


=== Top 15 des profils de jobs les plus fréquents ===


,reference,couleur,nombre_jobs,longueur_moyenne,nombre_sessions,nombre_centres_usinage
0,6112,654,594,6166.37,28,2
1,6101,654,413,6185.52,29,2
2,6107,654,272,6123.97,26,2
3,6121,654,220,5967.76,23,2
4,76281,WS,213,6169.49,10,2
5,6115,654,194,5958.76,25,2
6,6105,654,191,6092.32,26,2
7,6113,654,124,5872.77,28,2
8,141015,ALU,114,6237.32,24,2
9,141015,654,113,4875.07,23,2


In [9]:
# 6. Analyse de la production horaire
query6 = """
WITH ProductionHoraire AS (
    SELECT 
        cu.nom as centre_usinage,
        sp.date_production,
        EXTRACT(HOUR FROM pp.timestamp_production) as heure,
        COUNT(*) as pieces_produites
    FROM centre_usinage cu
    JOIN session_production sp ON cu.id = sp.centre_usinage_id
    JOIN piece_production pp ON sp.id = pp.session_id
    GROUP BY cu.nom, sp.date_production, EXTRACT(HOUR FROM pp.timestamp_production)
)
SELECT 
    centre_usinage,
    heure,
    ROUND(AVG(pieces_produites)::numeric, 2) as moyenne_pieces_par_heure,
    MAX(pieces_produites) as max_pieces_par_heure,
    COUNT(*) as nombre_jours_production
FROM ProductionHoraire
GROUP BY centre_usinage, heure
ORDER BY centre_usinage, heure;
"""

print("\n=== Analyse de la production horaire par centre d'usinage ===")
df6 = pd.read_sql(query6, engine)
df6


=== Analyse de la production horaire par centre d'usinage ===


,centre_usinage,heure,moyenne_pieces_par_heure,max_pieces_par_heure,nombre_jours_production
0,DEM12,5.0,39.30,62,20
1,DEM12,6.0,42.30,69,20
2,DEM12,7.0,48.37,68,19
3,DEM12,8.0,37.37,68,19
4,DEM12,9.0,43.12,60,17
5,DEM12,10.0,35.12,54,17
6,DEM12,11.0,41.24,77,17
7,DEM12,12.0,11.85,22,13
8,DEM12,13.0,17.59,31,17
9,DEM12,14.0,45.60,71,15


In [10]:
# 7. Analyse des commandes de volets roulants par type de coffre
query7 = """
SELECT 
    COALESCE(coffre, 'Non spécifié') as type_coffre,
    COUNT(*) as nombre_commandes,
    SUM(gestion_en_stock) as total_en_stock,
    COUNT(DISTINCT extension) as nombre_extensions_differentes,
    STRING_AGG(DISTINCT extension, ', ' ORDER BY extension) as extensions
FROM commandes_volets_roulants
GROUP BY coffre
ORDER BY nombre_commandes DESC;
"""

print("\n=== Analyse des commandes par type de coffre ===")
df7 = pd.read_sql(query7, engine)
df7


=== Analyse des commandes par type de coffre ===


,type_coffre,nombre_commandes,total_en_stock,nombre_extensions_differentes,extensions
0,SOP ALU H30,3,3,2,"A01, H00"
1,SOP ALU GAL,2,2,1,YG0
2,S D MODULAIRE,1,0,1,T20
3,SOP ALU A30-F21,1,0,1,Y00
4,SOP ALU MOTOR,1,1,1,D04
5,SOP ALU N30,1,0,1,000
6,SOP ALU PREM,1,1,1,F06
7,SOP CONNECTE,1,1,1,V01
8,SOP DURABLE,1,1,1,Q17
9,SOP ECO STAND,1,1,1,S19


In [11]:
# 8. Analyse des temps d'attente vs temps de production
query8 = """
SELECT 
    cu.nom as centre_usinage,
    COUNT(DISTINCT sp.id) as nombre_sessions,
    ROUND(AVG(sp.temps_production_effectif)::numeric, 2) as temps_production_moyen_heures,
    ROUND(AVG(sp.temps_attente)::numeric, 2) as temps_attente_moyen_heures,
    ROUND(AVG(sp.temps_arret_volontaire)::numeric, 2) as temps_arret_moyen_heures,
    ROUND(AVG(sp.taux_occupation)::numeric, 2) as taux_occupation_moyen,
    ROUND(AVG(sp.taux_attente)::numeric, 2) as taux_attente_moyen,
    ROUND(AVG(sp.taux_arret_volontaire)::numeric, 2) as taux_arret_moyen
FROM centre_usinage cu
JOIN session_production sp ON cu.id = sp.centre_usinage_id
GROUP BY cu.nom
ORDER BY nombre_sessions DESC;
"""

print("\n=== Analyse des temps de production et d'attente par centre d'usinage ===")
df8 = pd.read_sql(query8, engine)
df8


=== Analyse des temps de production et d'attente par centre d'usinage ===


,centre_usinage,nombre_sessions,temps_production_moyen_heures,temps_attente_moyen_heures,temps_arret_moyen_heures,taux_occupation_moyen,taux_attente_moyen,taux_arret_moyen
0,DEM12,20,7.85,2.13,3.38,59.88,15.41,24.72
1,DEMALU,20,7.49,1.36,3.39,62.74,10.28,26.98
2,SU12,20,10.03,2.26,1.89,71.47,15.76,12.77
